In [2]:
from dotenv import load_dotenv 
import os
import pymysql
import pandas as pd
import requests

load_dotenv()
class Config:
    MYSQL_HOST = os.getenv("MYSQL_HOST")
    MYSQL_PORT = int(os.getenv("MYSQL_PORT"))
    MYSQL_USER = os.getenv("MYSQL_USER")
    MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
    MYSQL_DB = os.getenv("MYSQL_DB")
    MYSQL_CHARSET = os.getenv("MYSQL_CHARSET")

connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)
# เชื่อมต่อกับ database

In [12]:
# query data from 2 tables

with connection.cursor() as cursor:
    cursor.execute("SELECT Book_ID, Price FROM audible_data")
    result = cursor.fetchall()
    cursor.execute("SELECT * FROM audible_transaction")
    result1 = cursor.fetchall()

In [15]:
# แปลงรูปแบบข้อมูลที่ query มาจากข้างบนจากรูปแบบ list ของ dictionary ให้อยู่ในรูปแบบ PandasDataFrame

audible_data = pd.DataFrame(result)
audible_data = audible_data.set_index('Book_ID')
audible_transaction = pd.DataFrame(result1)
print(audible_data)

           Price
Book_ID         
1        $29.65 
2        $24.95 
3        $41.99 
4        $29.65 
5        $29.65 
...          ...
2265     $14.95 
2266     $14.95 
2267     $24.95 
2268     $30.09 
2269     $35.71 

[2269 rows x 1 columns]


In [17]:
# join 2 tables เข้าด้วยกัน

transaction = audible_transaction.merge(audible_data, how="left", left_on="book_id", right_on="Book_ID")
print(transaction)

                  timestamp   user_id  book_id                   country  \
0       2021-05-01 00:00:01  ad8eca41     1584                  Portugal   
1       2021-05-01 00:00:03  561b26c1      829  United States of America   
2       2021-05-01 00:00:04  81f149e5     1391                     Japan   
3       2021-05-01 00:00:07  4f218413     1586                    Taiwan   
4       2021-05-01 00:00:18  a4066781      300  United States of America   
...                     ...       ...      ...                       ...   
1998816 2021-07-31 23:59:47  72da1411      600                  Portugal   
1998817 2021-07-31 23:59:48  620c27c8      462                Montenegro   
1998818 2021-07-31 23:59:55  84fe88ab     1656  United States of America   
1998819 2021-07-31 23:59:57  fd6cc4fc      522  United States of America   
1998820 2021-07-31 23:59:59  9f4d5e3b      110  United States of America   

           Price  
0        $35.00   
1        $19.59   
2        $31.50   
3        $1

In [ ]:
# ใช้ REST API ในการแปลงอัตราแลกเปลี่ยนให้เป็นเงินบาท

url = "https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate"
r = requests.get(url)
result_conversion_rate = r.json()
print(result_conversion_rate)

In [21]:
# แปลงเป็น PandasDataFrame

conversion_rate = pd.DataFrame(result_conversion_rate)
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
print(conversion_rate)

           date  conversion_rate
0    2021-04-01           31.194
1    2021-04-02           31.290
2    2021-04-03           31.256
3    2021-04-04           31.244
4    2021-04-05           31.342
..          ...              ...
129  2021-08-08           33.395
130  2021-08-09           33.464
131  2021-08-10           33.460
132  2021-08-11           33.145
133  2021-08-12           33.084

[134 rows x 2 columns]


In [23]:
# แปลงทั้งสอง dataframe ให้เป็น date เหมือนกันเพื่อ join 

transaction['date'] = transaction['timestamp']
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
transaction.head()

,timestamp,user_id,book_id,country,Price,date
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,$35.00,2021-05-01
1,2021-05-01 00:00:03,561b26c1,829,United States of America,$19.59,2021-05-01
2,2021-05-01 00:00:04,81f149e5,1391,Japan,$31.50,2021-05-01
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,$19.95,2021-05-01
4,2021-05-01 00:00:18,a4066781,300,United States of America,$17.49,2021-05-01


In [28]:
# join 2 dataframe เข้าด้วยกัน และกำจัดเครื่องหมาย $ ออก จากนั้นจึงทำการคำนวณเป็นค่าเงินบาทออกมา และ drop column ทิ้ง

final_df = transaction.merge(conversion_rate, how="left", right_on="date", left_on="date")
final_df["Price"] = final_df.apply(lambda x: x["Price"].replace("$", ""), axis=1)
final_df["Price"] = final_df["Price"].astype(float)
final_df["THBPrice"] = final_df["Price"] * final_df["conversion_rate"]
final_df = final_df.drop("date", axis=1)
final_df.head()

,timestamp,user_id,book_id,country,Price,conversion_rate,THBPrice
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,35.00,31.14,1089.9000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,19.59,31.14,610.0326
2,2021-05-01 00:00:04,81f149e5,1391,Japan,31.50,31.14,980.9100
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,19.95,31.14,621.2430
4,2021-05-01 00:00:18,a4066781,300,United States of America,17.49,31.14,544.6386


In [29]:
final_df.to_csv("output.csv", index=False)